In [1]:
import pandas as pd
data_info = pd.read_csv('data.info')

In [2]:
data_info['label'].value_counts() / len(data_info)

0    0.955063
1    0.044937
Name: label, dtype: float64

In [3]:
unique_ids = pd.Series(data_info['gene_id'].unique())
len(unique_ids)

3852

In [4]:
train_gene_id = unique_ids.sample(int(len(unique_ids)*0.8), random_state = 4266)

In [5]:
data_info_train = data_info[data_info['gene_id'].isin(train_gene_id)]
data_info_valid = data_info[~data_info['gene_id'].isin(train_gene_id)]
data_info_train.shape[0]+data_info_valid.shape[0]==data_info.shape[0]

True

In [6]:
data_info_valid['label'].value_counts()/len(data_info_valid)

0    0.956529
1    0.043471
Name: label, dtype: float64

In [7]:
import json
jsonlist = []
with open('dataset0.json') as f:
    for jsonobj in f:
        jsonlist.append(json.loads(jsonobj))

In [8]:
from tqdm import tqdm

In [9]:
df_dict = {'transcript_id' : [], 'transcript_position' : [], '5-mers': [],'readings':[]}
for i in tqdm(jsonlist):
    transcript_id = list(i.keys())[0]
    transcript_position = list(i[transcript_id].keys())[0]
    five_mer = list(i[transcript_id][transcript_position].keys())[0]
    for j in  i[transcript_id][transcript_position][five_mer]:
        df_dict['transcript_id'].append(transcript_id)
        df_dict['transcript_position'].append(transcript_position)
        df_dict['5-mers'].append(five_mer)
        df_dict['readings'].append(j)


  0%|          | 0/121838 [00:00<?, ?it/s]

100%|██████████| 121838/121838 [00:05<00:00, 23393.54it/s]


In [65]:
df  = pd.DataFrame(df_dict)

In [66]:
df

,transcript_id,transcript_position,5-mers,readings
0,ENST00000000233,244,AAGACCA,"[0.00299, 2.06, 125.0, 0.0177, 10.4, 122.0, 0...."
1,ENST00000000233,244,AAGACCA,"[0.00631, 2.53, 125.0, 0.00844, 4.67, 126.0, 0..."
2,ENST00000000233,244,AAGACCA,"[0.00465, 3.92, 109.0, 0.0136, 12.0, 124.0, 0...."
3,ENST00000000233,244,AAGACCA,"[0.00398, 2.06, 125.0, 0.0083, 5.01, 130.0, 0...."
4,ENST00000000233,244,AAGACCA,"[0.00664, 2.92, 120.0, 0.00266, 3.94, 129.0, 0..."
...,...,...,...,...
11027101,ENST00000641834,1693,TTGACAT,"[0.00418, 7.49, 108.0, 0.00564, 10.2, 116.0, 0..."
11027102,ENST00000641834,1693,TTGACAT,"[0.00664, 1.91, 109.0, 0.00598, 12.3, 110.0, 0..."
11027103,ENST00000641834,1693,TTGACAT,"[0.00721, 4.58, 105.0, 0.00398, 6.58, 113.0, 0..."
11027104,ENST00000641834,1693,TTGACAT,"[0.00266, 2.33, 109.0, 0.00913, 10.4, 108.0, 0..."


In [67]:
df[[f"readings_{i}" for i in range(9)]] = pd.DataFrame(df['readings'].tolist(), index = df.index)

In [68]:
df.drop(columns = 'readings', inplace = True)


In [69]:
new_columns = ['dwell_time_-1','sd_-1','mean_-1','dwell_time_0','sd_0','mean_0','dwell_time_1','sd_1','mean_1']
df = df.rename(columns=dict(zip(list(df.columns)[3:], new_columns)))

In [70]:
df.columns

Index(['transcript_id', 'transcript_position', '5-mers', 'dwell_time_-1',
       'sd_-1', 'mean_-1', 'dwell_time_0', 'sd_0', 'mean_0', 'dwell_time_1',
       'sd_1', 'mean_1'],
      dtype='object')

In [71]:
df

,transcript_id,transcript_position,5-mers,dwell_time_-1,sd_-1,mean_-1,dwell_time_0,sd_0,mean_0,dwell_time_1,sd_1,mean_1
0,ENST00000000233,244,AAGACCA,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1
1,ENST00000000233,244,AAGACCA,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9
2,ENST00000000233,244,AAGACCA,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6
3,ENST00000000233,244,AAGACCA,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4
4,ENST00000000233,244,AAGACCA,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2
...,...,...,...,...,...,...,...,...,...,...,...,...
11027101,ENST00000641834,1693,TTGACAT,0.00418,7.49,108.0,0.00564,10.20,116.0,0.01000,2.01,76.4
11027102,ENST00000641834,1693,TTGACAT,0.00664,1.91,109.0,0.00598,12.30,110.0,0.01760,2.61,74.6
11027103,ENST00000641834,1693,TTGACAT,0.00721,4.58,105.0,0.00398,6.58,113.0,0.00316,2.28,85.3
11027104,ENST00000641834,1693,TTGACAT,0.00266,2.33,109.0,0.00913,10.40,108.0,0.00664,4.44,76.8


In [72]:
df = df.drop(columns = [i for i in df.columns if 'weight' in i])

In [73]:
df['product_mean_dwell_-1'] = df['dwell_time_-1']*df['mean_-1']
df['product_mean_dwell_0'] = df['dwell_time_0']*df['mean_0']
df['product_mean_dwell_1'] = df['dwell_time_1']*df['mean_1']

In [74]:
df['product_var_dwell_-1'] = df['sd_-1']*df['sd_-1']*(df['dwell_time_-1'])
df['product_var_dwell_0'] = df['sd_0']*df['sd_0']*(df['dwell_time_0'])
df['product_var_dwell_1'] = df['sd_1']*df['sd_1']*(df['dwell_time_1'])

In [75]:
df

,transcript_id,transcript_position,5-mers,dwell_time_-1,sd_-1,mean_-1,dwell_time_0,sd_0,mean_0,dwell_time_1,sd_1,mean_1,product_mean_dwell_-1,product_mean_dwell_0,product_mean_dwell_1,product_var_dwell_-1,product_var_dwell_0,product_var_dwell_1
0,ENST00000000233,244,AAGACCA,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1,0.37375,2.15940,0.782130,0.012688,1.914432,1.104933
1,ENST00000000233,244,AAGACCA,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9,0.78875,1.06344,0.833270,0.040390,0.184067,0.408807
2,ENST00000000233,244,AAGACCA,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6,0.50685,1.68640,0.396408,0.071454,1.958400,0.022594
3,ENST00000000233,244,AAGACCA,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4,0.49750,1.07900,0.400392,0.016890,0.208331,0.071156
4,ENST00000000233,244,AAGACCA,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2,0.79680,0.34314,1.068600,0.056615,0.041293,0.664593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11027101,ENST00000641834,1693,TTGACAT,0.00418,7.49,108.0,0.00564,10.20,116.0,0.01000,2.01,76.4,0.45144,0.65424,0.764000,0.234498,0.586786,0.040401
11027102,ENST00000641834,1693,TTGACAT,0.00664,1.91,109.0,0.00598,12.30,110.0,0.01760,2.61,74.6,0.72376,0.65780,1.312960,0.024223,0.904714,0.119893
11027103,ENST00000641834,1693,TTGACAT,0.00721,4.58,105.0,0.00398,6.58,113.0,0.00316,2.28,85.3,0.75705,0.44974,0.269548,0.151240,0.172320,0.016427
11027104,ENST00000641834,1693,TTGACAT,0.00266,2.33,109.0,0.00913,10.40,108.0,0.00664,4.44,76.8,0.28994,0.98604,0.509952,0.014441,0.987501,0.130898


In [76]:
df['count'] = 1

In [77]:
df.dtypes

transcript_id             object
transcript_position       object
5-mers                    object
dwell_time_-1            float64
sd_-1                    float64
mean_-1                  float64
dwell_time_0             float64
sd_0                     float64
mean_0                   float64
dwell_time_1             float64
sd_1                     float64
mean_1                   float64
product_mean_dwell_-1    float64
product_mean_dwell_0     float64
product_mean_dwell_1     float64
product_var_dwell_-1     float64
product_var_dwell_0      float64
product_var_dwell_1      float64
count                      int64
dtype: object

In [78]:
new_df = df.groupby(['transcript_id','transcript_position','5-mers']).agg({'dwell_time_-1':'sum','dwell_time_0':'sum','dwell_time_1':'sum',
                                                                 'product_mean_dwell_-1':'sum','product_mean_dwell_0':'sum','product_mean_dwell_1':'sum',
                                                                 'product_var_dwell_-1':'sum','product_var_dwell_0':'sum','product_var_dwell_1':'sum','count':'sum'}).reset_index()

In [79]:
new_df

,transcript_id,transcript_position,5-mers,dwell_time_-1,dwell_time_0,dwell_time_1,product_mean_dwell_-1,product_mean_dwell_0,product_mean_dwell_1,product_var_dwell_-1,product_var_dwell_0,product_var_dwell_1,count
0,ENST00000000233,244,AAGACCA,1.52891,1.73405,1.35881,189.222290,218.132190,109.758379,41.489794,125.613831,43.932458,185
1,ENST00000000233,261,CAAACTG,1.13679,1.17182,1.32617,124.961054,126.675831,124.803983,17.380216,16.350451,15.720354,172
2,ENST00000000233,316,GAAACAG,1.40045,1.37187,1.39771,147.678850,136.399778,124.829067,16.195712,20.977549,8.747308,185
3,ENST00000000233,332,AGAACAT,2.12405,1.72646,1.22030,275.168090,168.921707,108.725012,90.916607,16.938959,6.869338,200
4,ENST00000000233,368,AGGACAA,2.11870,2.27289,1.98376,250.467580,277.123690,168.612070,118.886742,88.055792,40.463367,198
...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,610,AGAACTG,0.57474,0.38901,0.41809,74.009770,39.192241,38.475738,46.934373,7.329421,3.092440,68
121834,ENST00000641834,697,GTGACCT,0.42028,0.52584,0.39422,40.835807,63.796090,31.191882,10.116098,30.257552,6.057253,69
121835,ENST00000641834,864,TGGACAG,0.37512,0.43364,0.28434,42.833570,48.382040,21.193316,5.946455,20.485009,5.993002,44
121836,ENST00000641834,926,AGGACAT,0.82279,0.87808,0.56834,95.252400,101.595870,43.652918,31.391393,35.013854,3.831191,70


In [80]:
new_df['mean_dwell_time_-1'] = new_df['dwell_time_-1']/new_df['count']
new_df['mean_dwell_time_0'] = new_df['dwell_time_0']/new_df['count']
new_df['mean_dwell_time_1'] = new_df['dwell_time_1']/new_df['count']

In [81]:
new_df['weighted_mean_-1'] = new_df['product_mean_dwell_-1']/new_df['dwell_time_-1']
new_df['weighted_mean_0']= new_df['product_mean_dwell_0']/new_df['dwell_time_0']
new_df['weighted_mean_1']= new_df['product_mean_dwell_1']/new_df['dwell_time_1']

In [82]:
import numpy as np
new_df['weighted_sd_-1'] = np.sqrt(new_df['product_var_dwell_-1']/(new_df['dwell_time_-1']))
new_df['weighted_sd_0']= np.sqrt(new_df['product_var_dwell_0']/(new_df['dwell_time_0']))
new_df['weighted_sd_1']= np.sqrt(new_df['product_var_dwell_1']/(new_df['dwell_time_1']))

In [83]:
new_df.columns

Index(['transcript_id', 'transcript_position', '5-mers', 'dwell_time_-1',
       'dwell_time_0', 'dwell_time_1', 'product_mean_dwell_-1',
       'product_mean_dwell_0', 'product_mean_dwell_1', 'product_var_dwell_-1',
       'product_var_dwell_0', 'product_var_dwell_1', 'count',
       'mean_dwell_time_-1', 'mean_dwell_time_0', 'mean_dwell_time_1',
       'weighted_mean_-1', 'weighted_mean_0', 'weighted_mean_1',
       'weighted_sd_-1', 'weighted_sd_0', 'weighted_sd_1'],
      dtype='object')

In [84]:
new_df = new_df.drop(columns = ['dwell_time_-1',
       'dwell_time_0', 'dwell_time_1', 'product_mean_dwell_-1',
       'product_mean_dwell_0', 'product_mean_dwell_1', 'product_var_dwell_-1',
       'product_var_dwell_0', 'product_var_dwell_1', 'count'])

In [85]:
new_df

,transcript_id,transcript_position,5-mers,mean_dwell_time_-1,mean_dwell_time_0,mean_dwell_time_1,weighted_mean_-1,weighted_mean_0,weighted_mean_1,weighted_sd_-1,weighted_sd_0,weighted_sd_1
0,ENST00000000233,244,AAGACCA,0.008264,0.009373,0.007345,123.762870,125.793483,80.775369,5.209304,8.511144,5.686085
1,ENST00000000233,261,CAAACTG,0.006609,0.006813,0.007710,109.924484,108.101783,94.108586,3.910096,3.735377,3.442957
2,ENST00000000233,316,GAAACAG,0.007570,0.007416,0.007555,105.450998,99.426169,89.309704,3.400684,3.910397,2.501662
3,ENST00000000233,332,AGAACAT,0.010620,0.008632,0.006101,129.548782,97.842815,89.096953,6.542432,3.132313,2.372598
4,ENST00000000233,368,AGGACAA,0.010701,0.011479,0.010019,118.217577,121.925694,84.996204,7.490865,6.224289,4.516338
...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,610,AGAACTG,0.008452,0.005721,0.006148,128.770870,100.748672,92.027406,9.036699,4.340647,2.719667
121834,ENST00000641834,697,GTGACCT,0.006091,0.007621,0.005713,97.163336,121.322246,79.123033,4.906109,7.585603,3.919842
121835,ENST00000641834,864,TGGACAG,0.008525,0.009855,0.006462,114.186314,111.571903,74.535120,3.981475,6.873112,4.590957
121836,ENST00000641834,926,AGGACAT,0.011754,0.012544,0.008119,115.767571,115.702294,76.807752,6.176761,6.314703,2.596347


In [86]:
new_df.to_csv('weighted_summary.csv')

In [88]:
import pandas as pd
new_df = pd.read_csv('weighted_summary.csv')

In [89]:
new_df.columns

Index(['Unnamed: 0', 'transcript_id', 'transcript_position', '5-mers',
       'mean_dwell_time_-1', 'mean_dwell_time_0', 'mean_dwell_time_1',
       'weighted_mean_-1', 'weighted_mean_0', 'weighted_mean_1',
       'weighted_sd_-1', 'weighted_sd_0', 'weighted_sd_1'],
      dtype='object')

In [90]:
new_df = new_df.drop(columns = {'Unnamed: 0'})

In [91]:
new_df['5-mer-0'] = new_df['5-mers'].map(lambda x:x[0])
new_df['5-mer-1'] = new_df['5-mers'].map(lambda x:x[1])
new_df['5-mer-2'] = new_df['5-mers'].map(lambda x:x[2])
new_df['5-mer-3'] = new_df['5-mers'].map(lambda x:x[3])
new_df['5-mer-4'] = new_df['5-mers'].map(lambda x:x[4])
new_df['5-mer-5'] = new_df['5-mers'].map(lambda x:x[5])
new_df['5-mer-6'] = new_df['5-mers'].map(lambda x:x[6])

In [92]:
new_df['5-mer-4'].value_counts()

C    121838
Name: 5-mer-4, dtype: int64

In [93]:
new_df=new_df.drop(columns = ['5-mers','5-mer-3','5-mer-4'])

In [94]:
new_df_cat = new_df[['5-mer-0','5-mer-1','5-mer-2','5-mer-5','5-mer-6']]

In [95]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
one_hot_encoded = encoder.fit_transform(new_df_cat)

In [96]:
one_hot_encoded_df = pd.DataFrame(one_hot_encoded.toarray(), columns=encoder.get_feature_names_out(input_features=new_df_cat.columns))

In [97]:
data_encoded = pd.concat([new_df.drop(columns=new_df_cat.columns), one_hot_encoded_df], axis=1)

In [98]:
df_train = data_encoded[data_encoded['transcript_id'].isin(data_info_train['transcript_id'])]
df_valid = data_encoded[~data_encoded['transcript_id'].isin(data_info_train['transcript_id'])]

In [99]:
df_train['transcript_position'] = df_train['transcript_position'].astype(int)
df_train = pd.merge(df_train, data_info_train, on = ['transcript_id','transcript_position'])

C:\Users\xghan\AppData\Local\Temp\ipykernel_11728\3048243875.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['transcript_position'] = df_train['transcript_position'].astype(int)


In [100]:
df_valid['transcript_position'] = df_valid['transcript_position'].astype(int)
df_valid = pd.merge(df_valid, data_info_valid,on=["transcript_id","transcript_position"])

C:\Users\xghan\AppData\Local\Temp\ipykernel_11728\286692896.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid['transcript_position'] = df_valid['transcript_position'].astype(int)


In [101]:
df_valid.shape[0]+df_train.shape[0] == new_df.shape[0]

True

In [102]:
df_valid.to_csv('label_df_valid.csv', index = False)
df_train.to_csv('label_df_train.csv', index = False)